In [ ]:
import numpy as np
import pandas as pd
import time
import random
import warnings
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from scipy.stats import truncnorm, norm, multivariate_normal

## Q4

In [ ]:
def gibbs(K, m1, m2, s1, s2, st, y0, t0, burn_in):
    '''
    implment Gibbs sampling
    
    Parameters:
    K: the amount of samples
    t0: initial outcome 
    
    Return:
    s1List, s2List: samples from conditional distribution of skills
    t: estimated outcome 
    time: run time
    '''
    s1List = np.zeros(K)
    s2List = np.zeros(K)
    t = np.zeros(K)
    t[0] = t0
    
    for k in range(K-1):
        if k == burn_in:
            timer_0 = time.perf_counter()
        s1List[k+1], s2List[k+1] = pdf_s_given_t_rvs(m1, m2, s1, s2, st, t[k])
        t[k+1] = pdf_t_given_s_y_rvs(s1List[k], s2List[k], st, y0)
    return s1List, s2List, t, time.perf_counter() - timer_0

def pdf_s_given_t_rvs(m1, m2, s1, s2, st, t):
    '''
    compute conditional distribution of skills p(s|t,y)
    
    Parameters:
    m1,m2,s1,s2,st: mean and std
    t: outcome
    
    Return:
    rv: random variable of conditional distribution
    '''    
    M = np.array([[1,-1]])  
    cov_s_given_t = np.linalg.inv(np.diag([1/s1**2, 1/s2**2]) + 1/st**2 * M.T.dot(M))
    cov_s_inv = np.linalg.inv(np.diag([s1**2, s2**2]))
    m_s = np.array([[m1],[m2]])
    m_s_given_t = cov_s_given_t.dot(cov_s_inv.dot(m_s) + M.T/st**2 * t)
    rv = multivariate_normal.rvs(mean = m_s_given_t.flatten(), cov = cov_s_given_t)     
    return rv


In [ ]:
def pdf_t_given_s_y_rvs(s1List,s2List,st,y):
    '''
    compute full conditional distribution of skills p(t|s1,s2,y), truncated normal distribution TN(t;s1-s2,st^2,y)
    
    Parameters:
    s1List, s2List: samples from conditional distribution
    st: std of the full conditional distribution of the outcome
    y: win situation of player1
    
    Return:
    rv: random variable of full conditional distribution of skills
    '''
    mu = s1List - s2List
    if y == 1:
        return truncnorm.rvs(-mu/st, np.inf, scale = st) + mu
    else:
        return truncnorm.rvs(-np.inf, -mu/st, scale = st) + mu

def posterior(s1Gibbs, s2Gibbs):
    '''
    compute posterior distribution N(mu, s**2) where mu, s**2 comes from Gibbs sampling
    
    Parameters: 
    s1Gibbs, s2Gibbs: post-burn_in samples for two players using Gibbs Sampling
    
    Return:
    posterior normal distribution for two players such as rvs1, rvs2
    '''
    return norm.freeze(loc = np.mean(s1Gibbs),scale = np.std(s1Gibbs)), norm.freeze(loc = np.mean(s2Gibbs),scale = np.std(s2Gibbs))

In [ ]:
# initial setting
m1 = 25
m2 = 25
s1 = 25/3
s2 = 25/3
st = 25/3
y = 1
t0 = 200

## Q4.1

In [ ]:
K = 200
burn_in = 20
s_1, s_2, t, runtime = gibbs(K, m1, m2, s1, s2, st, y, t0, burn_in)
s_1_post, s_2_post = posterior(s_1[burn_in:], s_2[burn_in:])

plt.plot(s_1, label='s1')
plt.plot(s_2, label='s2')
plt.xlabel('samples')
plt.legend()
plt.ylim(-50,100)
plt.grid()
plt.savefig('4.1Posterior_Gibbs.png')

## Q4.3

In [ ]:
def plot_gibbs(k):
    '''
    plot the histogram of post-burn_in samples and corresponding Gaussian dist. approximation. 
    Save the resulting figure on the working directory
    
    Parameters:
    k: the amount of samples, including burn_in  
    '''
    s1List,s2List,t,runtime = gibbs(k,m1,m2,s1,s2,st, y,t0,burn_in)
    s1Post, s2Post = posterior(s1List[burn_in::], s2List[burn_in::])
    
    x_eval = np.linspace(10,50,200)
    plt.title('# samples = ' + str(k-burn_in) + ', time: ' + str(np.round(runtime,3)) + 's', fontsize = 8)
    plt.hist(s1List[burn_in::],density = 1, bins = 50)
    plt.plot(x_eval, s1Post.pdf(x_eval))
    plt.ylim(0,0.1)
    plt.xlim(10,50)
    plt.xlabel('s1')
    plt.ylabel('p(s1|y=1)')
    plt.savefig('4.3_K=' + str(k-burn_in) + '.png')
    plt.clf()

In [ ]:
plot_gibbs(100)
plot_gibbs(200)
plot_gibbs(500)
plot_gibbs(1000)

## Q4.2/Q4.4

In [ ]:
K = 200
burn_in = 20
t0 = 200
s_1, s_2, _, _ = gibbs(K, m1, m2, s1, s2, st, y, t0, burn_in)
x1PostGibbs, s1PostGibbs = np.mean(s_1[burn_in:]), np.std(s_1[burn_in:])
x2PostGibbs, s2PostGibbs = np.mean(s_2[burn_in:]), np.std(s_2[burn_in:])
x  = np.linspace(-10,60,1000)
plt.plot(x, norm.pdf(x,x1PostGibbs,s1PostGibbs),label = 'Winner,s1')
plt.plot(x, norm.pdf(x,x2PostGibbs,s2PostGibbs),label = 'Loser,s2') 
plt.plot(x, norm.pdf(x,m1,s1), '--', label = 'Prior')
plt.xlabel(r"$s_1, s_2$")
plt.ylabel(r"$p(s_1|y=1)/p(s_2|y=1)$")
plt.legend()
plt.grid()
plt.savefig('4.2Prior_posterior.png')

## Q5

In [ ]:
def generateRank(randomOrder = False):
    '''
    use posterior distribution as a prior for the next match to estimate teams' skill
    by Gibbs sampling. Save the result as dataframe in working directory
    
    Parameter:
    randomOrder: boolean, if re-arrange the order of match in SerieA
    
    Return:
    sortedfinal: dataframe, skill information for each team and its rank
    '''
    serieA = pd.read_csv('SerieA.csv')
    team = serieA.team1.unique()
    n_team = len(team)
    matchTeams = list(zip(serieA.team1, serieA.team2, serieA.score1 - serieA.score2))

    if randomOrder:
        random.shuffle(matchTeams)

    n_match = len(serieA)
    mu = 25
    std = 25/3
    teamDict = {t:[(mu,std)] for t in team}
    st = 25/3
    burn_in = 20
    k = 500

    for t1, t2, t in matchTeams:
        m1,s1 = teamDict[t1][-1]
        m2,s2 = teamDict[t2][-1]
        y = np.sign(t)

        if t != 0:
            # if it is not draw
            s1List, s2List,_,_ = gibbs(k, m1, m2, s1, s2, st, y, t, burn_in)
            # compute mean and std from Gibbs Samplers
            mu1_new = np.mean(s1List[burn_in:])
            mu2_new = np.mean(s2List[burn_in:])
            s1_new = np.std(s1List[burn_in:])
            s2_new = np.std(s2List[burn_in:])
        else:
            # if it is a draw then repeat the previous value
            mu1_new = m1
            mu2_new = m2
            s1_new = s1
            s2_new = s2
        teamDict[t1].append((mu1_new, s1_new))
        teamDict[t2].append((mu2_new, s2_new))
        
    final = {}
    for team in teamDict.keys():
        finalmu = teamDict[team][-1][0]
        finalstd =  teamDict[team][-1][1]
        final[team] = np.round([finalmu, finalstd, finalmu - 3*finalstd],2)
    finaldf = pd.DataFrame.from_dict(final, orient = 'index', columns = ['mean','std','skillEstimate'])
    finaldf['rank'] = finaldf['mean'].rank(ascending = False)
    sortedfinal = finaldf.sort_values('rank')
    sortedfinal.to_csv(str(randomOrder) + 'rank.csv')
    return sortedfinal

In [ ]:
sortedfinal = generateRank()

In [ ]:
sortedfinal

In [ ]:
sortedfinal = generateRank(randomOrder = True)

In [ ]:
sortedfinal

## Q7

In [ ]:
serieA = pd.read_csv('SerieA.csv')
team = serieA.team1.unique()
n_team = len(team)
matchTeams = list(zip(serieA.team1, serieA.team2, serieA.score1 - serieA.score2))

n_match = len(serieA)
mu = 25
std = 25/3
teamDict = {t:[(mu,std)] for t in team}
st = 25/3
burn_in = 20
k = 500

meanPred = 0
estimatePred = 0
withoutDrawGame = 0

for t1, t2, t in matchTeams:
    m1,s1 = teamDict[t1][-1]
    m2,s2 = teamDict[t2][-1]
    y = np.sign(t)
    
    if t != 0:
        # if it is not draw
        s1List, s2List,_,_ = gibbs(k, m1, m2, s1, s2, st, y, t, burn_in)
        # compute mean and std from Gibbs Samplers
        mu1_new = np.mean(s1List[burn_in:])
        mu2_new = np.mean(s2List[burn_in:])
        s1_new = np.std(s1List[burn_in:])
        s2_new = np.std(s2List[burn_in:])
        
        # use mean and estimate for prediction, respectively
        meanPred += (np.sign(m1-m2) == y)
        estimatePred += (np.sign((m1-s1*3) - (m2-s2*3)) == y)
        withoutDrawGame += 1
    else:
        # if it is a draw then repeat the previous value
        mu1_new = m1
        mu2_new = m2
        s1_new = s1
        s2_new = s2    
       
    teamDict[t1].append((mu1_new, s1_new))
    teamDict[t2].append((mu2_new, s2_new))

In [ ]:
print('Considering Draw and using mean for prediction, the correct guess rate is {:.2f}%'.format(meanPred/n_match*100))
print('Considering Draw and using estimate for prediction, the correct guess rate is {:.2f}%'.format(estimatePred/n_match*100))
print('Disregarding Draw and using mean for prediction, the correct guess rate is {:.2f}%'.format(meanPred/withoutDrawGame*100))
print('Disregarding Draw and using estimate for prediction, the correct guess rate is {:.2f}%'.format(estimatePred/withoutDrawGame*100))

## Q9

In [ ]:
def truncGaussMM(lower, upper, m, s):
    '''
    computes the mean and variance of a truncated Gaussian distribution
    
    Parameters:
    lower, upper: The interval [lower, upper on which the Gaussian is being truncated
    m,s: mean and variance of the Gaussian which is to be truncated
    
    Return:
    trunc_mu, trunc_s: mean and variance of the truncated Gaussian
    '''
    lower_scale = (lower - m) / np.sqrt(s)
    upper_scale = (upper - m) / np.sqrt(s)
    trunc_mu = truncnorm.mean(lower_scale, upper_scale, loc = m, scale = np.sqrt(s))
    trunc_s = truncnorm.var(lower_scale, upper_scale, loc= m, scale = np.sqrt(s))
    return trunc_mu, trunc_s

In [ ]:
def divideGauss(m1, m2, s1, s2):
    '''
    computes the Gaussian distribution N(m,s) being propotional to N(m1,s1)/N(m2,s2)
    
    Parameters:
    m1, s1: mean and variance of the numerator Gaussian
    m2, s2: mean and variance of the denominator Gaussian

    Return:
    m, s: mean and variance of the quotient Gaussian
    '''
    m,s = multiplyGauss(m1,m2,s1,s2)
    return m, s

def multiplyGauss(m1, m2, s1, s2):
    '''
    computes the Gaussian distribution N(m,s) being propotional to N(m1,s1)*N(m2,s2)
    
    Parameters:
    m1, m2, s1, s2: mean and variance of the first and second Gaussian distribution
    
    Return:
    m,s = mean and variance of the product Gaussian
    '''
    s = 1/(1/s1+1/s2)
    m = (m1/s1+m2/s2)*s
    return m, s

In [ ]:
def message_passing(y, m1, m2, v1, v2, vt):
    '''
    return posterior distribution of skills for players using message passing algorithm
    '''
    # message from factor f1 to node s1
    mu_f1_s1_m = m1
    mu_f1_s1_v = v1
    # message from factor f2 to node s1
    mu_f2_s2_m = m2
    mu_f2_s2_v = v2

    # message from nodes s1 to factor f_sw
    mu_s1_sw_m = mu_f1_s1_m
    mu_s1_sw_v = mu_f1_s1_v
    # message from nodes s2 to factor f_sw
    mu_s2_sw_m = mu_f2_s2_m
    mu_s2_sw_v = mu_f2_s2_v
    
    # message from factor f_sw to node w
    mu_sw_w_m = mu_f1_s1_m- mu_f2_s2_m
    mu_sw_w_v= mu_f1_s1_v + mu_f2_s2_v
    
    # message from node w to factor f_wt
    mu_w_wt_m = mu_sw_w_m
    mu_w_wt_v = mu_sw_w_v
    
    # message from factor f_wt to node t
    mu_wt_t_m = mu_w_wt_m
    mu_wt_t_v = mu_w_wt_v + vt
    
    # Do moment matching of the marginal of t
    if y == 1:
        lower, upper = 0, 1000
    elif y == -1:
        lower, upper = -1000, 0
    # transform the truncated Gaussian distribution into a complete Gaussian distribution
    pt_m, pt_v = truncGaussMM(lower, upper, mu_wt_t_m, mu_wt_t_v)
    
    # Compute the message from factor f_ty to t
    mu_hat_ty_t_m, mu_hat_ty_t_v = divideGauss(pt_m, mu_wt_t_m,pt_v, mu_wt_t_v)
    
    # sending message backward
    # message from node t to factor f_wt
    mu_t_wt_m = mu_hat_ty_t_m
    mu_t_wt_v = mu_hat_ty_t_v
    
    # message from factor f_wt to node w
    mu_wt_t_m = mu_t_wt_m
    mu_wt_t_v = mu_t_wt_v + vt
    
    # message from node w to factor f_sw
    mu_w_sw_m = mu_wt_t_m
    mu_w_sw_v = mu_wt_t_v
    
    # message from factor f_sw to node s1
    mu_sw_s1_m = mu_f2_s2_m + mu_w_sw_m
    mu_sw_s1_v = mu_f2_s2_v + mu_w_sw_v 

    # message from factor f_sw to node s2
    mu_sw_s2_m =  mu_f1_s1_m  - mu_w_sw_m 
    mu_sw_s2_v =  mu_f1_s1_v + mu_w_sw_v 
    
    # compute the marginal of s1 and s2
    p_s1_m , p_s1_v = multiplyGauss(mu_f1_s1_m,mu_sw_s1_m,mu_f1_s1_v, mu_sw_s1_v)
    p_s2_m , p_s2_v = multiplyGauss(mu_f2_s2_m,mu_sw_s2_m,mu_f2_s2_v, mu_sw_s2_v)
    return p_s1_m , p_s1_v, p_s2_m , p_s2_v

In [ ]:
m1, m2 = 25, 25
v1, v2 = (25/3)**2,(25/3)**2
vt = (25/3)**2
y = 1
p_s1_m , p_s1_v, p_s2_m , p_s2_v = message_passing(y, m1, m2, v1, v2, vt)

# plot the posteriors computed with messsage passing
k = 500
burn_in = 20
t = 200
x = np.linspace(m1-v1, m1+v1, k-burn_in)
# draw values from Gaussian distribution

# generate pdf
s1_pdf = norm.pdf(x,p_s1_m,np.sqrt(p_s1_v))
s2_pdf = norm.pdf(x,p_s2_m,np.sqrt(p_s2_v))
s1List, s2List,_,_ = gibbs(k, m1, m2, np.sqrt(v1),np.sqrt(v2), np.sqrt(vt), y, t, burn_in)

s1Gibbs, s2Gibbs = s1List[burn_in:], s2List[burn_in:]
s1Gibbs_pdf = norm.pdf(x,np.mean(s1Gibbs),np.std(s1Gibbs))
s2Gibbs_pdf = norm.pdf(x,np.mean(s2Gibbs),np.std(s2Gibbs))

# plot the histogram and mp pdf
plt.hist(s1Gibbs,label='s1_Gibbs',density = True,bins=50, alpha = 0.4)
plt.plot(x,s1Gibbs_pdf ,'--',label = 's1_Gibbs_Gauss')
plt.plot(x,s1_pdf,'--',label = 's1_MP')
plt.legend()
plt.savefig('mp_gauss_s1.png')
plt.show()

In [ ]:
# plot the histogram and mp pdf
plt.hist(s2Gibbs,label='s2_Gibbs',density = True,bins=50, alpha = 0.4)
plt.plot(x,s2Gibbs_pdf ,'--',label = 's2_Gibbs_Gauss')
plt.plot(x,s2_pdf,'--',label = 's2_MP')
plt.legend()
plt.savefig('mp_gauss_s2.png')
plt.show()

## Q10

In [ ]:
def generateRank():
    '''
    use posterior distribution as a prior for the next match to estimate teams' skill
    by Gibbs sampling. Save the result as dataframe in working directory   
    
    Return:
    sortedfinal: dataframe, skill information for each team and its rank
    '''
    serieA = pd.read_csv('overwatch_2018_rs.csv')
    team = serieA.team1.unique()
    n_team = len(team)
    matchTeams = list(zip(serieA.team1, serieA.team2, serieA.score1 - serieA.score2))

    n_match = len(serieA)
    mu = 25
    std = 25/3
    teamDict = {t:[(mu,std)] for t in team}
    st = 25/3
    burn_in = 50
    k = 500

    for t1, t2, t in matchTeams:
        m1,s1 = teamDict[t1][-1]
        m2,s2 = teamDict[t2][-1]
        y = np.sign(t)

        if t != 0:
            # if it is not draw
            s1List, s2List,_,_ = gibbs(k, m1, m2, s1, s2, st, y, t, burn_in)
            # compute mean and std from Gibbs Samplers
            mu1_new = np.mean(s1List[burn_in:])
            mu2_new = np.mean(s2List[burn_in:])
            s1_new = np.std(s1List[burn_in:])
            s2_new = np.std(s2List[burn_in:])
        else:
            # if it is a draw then repeat the previous value
            mu1_new = m1
            mu2_new = m2
            s1_new = s1
            s2_new = s2
        teamDict[t1].append((mu1_new, s1_new))
        teamDict[t2].append((mu2_new, s2_new))
        
    final = {}
    for team in teamDict.keys():
        finalmu = teamDict[team][-1][0]
        finalstd =  teamDict[team][-1][1]
        final[team] = np.round([finalmu, finalstd, finalmu - 3*finalstd],2)
    finaldf = pd.DataFrame.from_dict(final, orient = 'index', columns = ['mean','std','skillEstimate'])
    finaldf['rank'] = finaldf['mean'].rank(ascending = False)
    sortedfinal = finaldf.sort_values('rank')
    sortedfinal.to_csv('overwatchrank.csv')
    return sortedfinal

In [ ]:
sortedfinal = generateRank()
sortedfinal['True Rank'] = ['New York Excelsior','Los Angeles Valiant','Boston Uprising','Los Angeles Gladiators'
                            ,'London Spitfire','Philadelphia Fusion','Houston Outlaws','Seoul Dynast','San Francisco Shock'
                           ,'Dallas Fuel','Florida Mayhem','Shanghai Dragons']
sortedfinal.to_csv('overwatchrank.csv')
sortedfinal

In [ ]:
serieA = pd.read_csv('overwatch_2018_rs.csv')
team = serieA.team1.unique()
n_team = len(team)
matchTeams = list(zip(serieA.team1, serieA.team2, serieA.score1 - serieA.score2))

n_match = len(serieA)
mu = 25
std = 25/3
teamDict = {t:[(mu,std)] for t in team}
st = 25/3
burn_in = 50
k = 500

meanPred = 0
estimatePred = 0

for t1, t2, t in matchTeams:
    m1,s1 = teamDict[t1][-1]
    m2,s2 = teamDict[t2][-1]
    y = np.sign(t)
    
    if t != 0:
        # if it is not draw
        s1List, s2List,_,_ = gibbs(k, m1, m2, s1, s2, st, y, t, burn_in)
        # compute mean and std from Gibbs Samplers
        mu1_new = np.mean(s1List[burn_in:])
        mu2_new = np.mean(s2List[burn_in:])
        s1_new = np.std(s1List[burn_in:])
        s2_new = np.std(s2List[burn_in:])
        
        # use mean and estimate for prediction, respectively
        meanPred += (np.sign(m1-m2) == y)
        estimatePred += (np.sign((m1-s1*3) - (m2-s2*3)) == y)   
       
    teamDict[t1].append((mu1_new, s1_new))
    teamDict[t2].append((mu2_new, s2_new))

In [ ]:
print('Using mean for prediction, the correct guess rate is {:.2f}%'.format(meanPred/n_match*100))
print('Using estimate for prediction, the correct guess rate is {:.2f}%'.format(estimatePred/n_match*100))

## Q11

In [ ]:
def run_mp_tuner(st, ordered=True):
    '''
        st is variance for game result t

        if ordered = 'True' or similar, the processing is done in chronological order
        if ordered = 'False', its the reversed.
    '''
    serie_A = pd.read_csv('overwatch_2018_rs.csv')
    team = list(serie_A.team1.unique())
    n_teams = len(team)

    t = np.array(serie_A.score1 - serie_A.score2)
    y = np.sign(t)
    M = len(y) 
    burn_in = 50
    K = 500
    
    team_dict = {}
    for team in team:
        team_dict[team] = list([[25], [25/3]])
    
    team1 = np.array(serie_A.team1)
    team2 = np.array(serie_A.team2)

    # initialize array for predictions, one element per match saying either -1 or +1
    prediction = []
    prediction_correct = [] # will have a value 1 in elements where the match was correctly predicted

    # for predictions with conservative skill estimate
    conservative_prediction = []
    conservative_prediction_correct = []



    if ordered == 1:
        range_play = range(M)
    elif ordered == 0:
        range_play = range(M-1, 0, -1)
    
    for m in range_play:
    # for m in range(80):
            # if m % 20 == 0:
            #     print(m, '/', M)
            # determine what teams are playing
            t1 = team1[m]
            t2 = team2[m]

            # extract means and std for both teams
            m1 = team_dict[t1][0][-1]
            s1 = team_dict[t1][1][-1]
            m2 = team_dict[t2][0][-1]
            s2 = team_dict[t2][1][-1]

            if t[m] != 0: # we skip matches that were draw and repeat the previous value instead
                # input outcome, means and variances
                m1_new, s1_new, m2_new, s2_new = message_passing(y[m], m1, s1**2, m2, s2**2, st)

                # prediction: t has mean s_1 - s_2 so we define the prediction as y = sgn(s_1 - s_2)
                y_prediction = np.sign(m1 - m2)
                y_cons_pred = np.sign((m1 - 3*s1) - (m2 - 3*s2))
                y_correct = np.sign(t[m])
                
                prediction.append(y_prediction)
                prediction_correct.append(y_correct == y_prediction)
                conservative_prediction.append(y_cons_pred)
                conservative_prediction_correct.append(y_correct == y_cons_pred)
            else:
                m1_new = m1
                s1_new = s1
                m2_new = m2
                s2_new = s2
            # update the dictionary for the two teams
            team_dict[t1][0].append(m1_new)
            team_dict[t1][1].append(s1_new)
            team_dict[t2][0].append(m2_new)
            team_dict[t2][1].append(s2_new)

    correct_prediction_rate = np.mean(prediction_correct)
    correct_conservative_prediction_rate = np.mean(conservative_prediction_correct)

    return correct_conservative_prediction_rate

In [ ]:
def run_tune_st_mp():
    print('Running tuner over sigma_t.\nShowing progress:')
    N = 20000
    std_grid = np.linspace(0.1, 5, num=N)

    r = []
    for c, std in enumerate(std_grid,0):
        if c % 100 == 0:
            print(c, '/', N, end='\r')
        r.append(run_mp_tuner(std**2))
    warnings.filterwarnings('ignore')
    plt.figure()
    plt.plot(std_grid**2, r)
    plt.xlabel(r'$\sigma_t^2$')
    plt.ylabel('Correct Prediction Rate')
    plt.grid()
    plt.figure()
    plt.plot(std_grid, r)
    plt.xlabel(r'$\sigma_t$')
    plt.ylabel('Correct Prediction Rate')
    plt.grid()
    print('Finished!')

In [ ]:
run_tune_st_mp()